In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [7]:
localidad = pd.read_csv("../Datasets/Localidades.csv",delimiter = ',',encoding = "utf-8")
cliente = pd.read_csv("../Datasets/Clientes.csv",encoding = "utf-8",delimiter = ';', usecols=["ID","Provincia","Nombre_y_Apellido","Domicilio","Telefono","Edad","Localidad","X","Y","col10"])
proveedor = pd.read_csv("../Datasets/Proveedores.csv",delimiter = ',',encoding = "ansi")
gasto = pd.read_csv("../Datasets/Gasto.csv",delimiter = ',',encoding = "utf-8")
compra = pd.read_csv("../Datasets/Compra.csv",delimiter = ',',encoding = "utf-8")
sucursal = pd.read_csv("../Datasets/Sucursales.csv",delimiter = ';',encoding = "utf-8")
venta = pd.read_csv('../Datasets/Venta.csv', delimiter= ',', encoding= "utf-8")


In [8]:
#Creo dataframe
provincias = pd.DataFrame()

### TABLA VENTA

In [9]:
def CargaArchivoVenta():
    #ds_path= pathlib.Path("Datasets")
    #print(ds_path)
    df_list=[]
    #archivo=glob.glob('.csv')
    #print("Hola")
    for file in glob.glob('Vent*.csv'):
        #print(file)
        df=pd.read_csv(file,delimiter = ',',encoding = "utf-8")
        df_list.append(df)
    venta=pd.concat(df_list)
    #Tratamiento outliers columna precio
    #Precio maximo de venta
    #venta['Precio'].max()

    Q1 = venta["Precio"].quantile(0.25)
    Q3 = venta["Precio"].quantile(0.75)
    #Se calcula el rango intercuartilico IQR.
    IQR = Q3 -Q1

    #Uso de mascara para filtrar
    outliersSup = (Q3 + (1.5*IQR))

    #Uso la mascara para filtrar
    mask = venta["Precio"]<outliersSup
    VentaSinOut2 = venta[mask]
    #VentaSinOut2.describe().T

    #Compruebo cantidad de outliers
    #venta.shape[0]-VentaSinOut2.shape[0]

    #Tratamiento de outliers
    Precio_Producto= VentaSinOut2.groupby(["IdProducto"])["Precio"].mean().reset_index()
    Precio_Producto= Precio_Producto.rename(columns= {"Precio":"Precio2"})
    venta = pd.merge(venta,Precio_Producto, how="left", on=["IdProducto"])
    venta.sort_values("IdVenta")

    #Se reemplazan los 0 de precios2 por el valor de la media de los precios sin outlier
    venta.Precio2.fillna(VentaSinOut2.Precio.mean(), inplace=True) #1065

    #venta.isnull().sum()

    #Se reemplazan con 0 y 1 todos los valores de precios vacios y cantidades
    venta.Precio.fillna(0, inplace= True)
    venta.Cantidad.fillna(1, inplace= True)

    #se crea columna con unos
    venta['Outliers'] = np.ones(venta.shape[0])

    #Se marcan con 0 las filas que son outliers para reemplazarlas con precios 2 despues
    venta.loc[(venta['Precio']>=outliersSup),'Outliers'] = 0

    #Este for reemplaza los 0 y los outliers por el precio promedio sin outliers
    for i in range(0,venta.shape[0]):
        if venta.Precio[i] == 0 or venta.Precio[i]> outliersSup:
            venta.Precio[i] = venta.Precio2[i]

    venta.drop(columns=['Precio2'], inplace=True)
    #venta['Total_venta'] = venta['Precio']*venta['Cantidad']

    #venta.isnull().sum()

    #tratamiento outliers columna cantidad
    Q1 = venta["Cantidad"].quantile(0.25)
    Q3 = venta["Cantidad"].quantile(0.75)

    #se calcula el rango intercuartilico IQR.
    IQR = Q3 -Q1
    outliersSup = (Q3 + (1.5*IQR)) 
    mask = venta["Cantidad"]<outliersSup
    ventaCantSinOut = venta[mask]
    cantidadMedia = round(ventaCantSinOut.Cantidad.mean())
    venta.loc[(venta['Cantidad'] >= outliersSup),"Cantidad"] = cantidadMedia #2

    #se crea la columna Total
    venta["Ventas_Totales"]= venta["Precio"]*venta["Cantidad"]

 
    venta.to_csv(r'C:/Users/notebook/OneDrive/Escritorio/Henry clase/DS-PI-ProyectoIndividual/Nuevosdataset/VentasNor.csv', index =False, header=True)

In [10]:
CargaArchivoVenta()

C:\Users\notebook\AppData\Local\Temp\ipykernel_6672\2595160031.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  venta.Precio[i] = venta.Precio2[i]


### TABLA COMPRAS 

In [11]:
def CargaArchivoCompra():
    #ds_path= pathlib.Path("Datasets")
    #print(ds_path)
    df_list=[]
    #archivo=glob.glob('.csv')
    #print("Hola")
    for file in glob.glob('Compr*.csv'):
        print(file)
        df=pd.read_csv(file,delimiter = ',',encoding = "utf-8")
        df_list.append(df)
    compra=pd.concat(df_list)
    #compra.head()
    #tratamiento outliers precio
    #Precio maximo de compra
    #compra['Precio'].max()

    Q1 = compra["Precio"].quantile(0.25)
    Q3 = compra["Precio"].quantile(0.75)
    #Se calcula el rango intercuartilico IQR.
    IQR = Q3 -Q1
    outliersSup = (Q3 + (1.5*IQR))

    #Uso de mascara para filtrar
    mask = compra["Precio"]<outliersSup
    CompraSinOut2 = compra[mask]
    #CompraSinOut2.describe().T

    #Compruebo cantidad de outliers
    #compra.shape[0]-CompraSinOut2.shape[0]

    #Tratamiento de outliers
    Precio_Producto= CompraSinOut2.groupby(["IdProducto"])["Precio"].mean().reset_index()
    Precio_Producto= Precio_Producto.rename(columns= {"Precio":"Precio2"})
    compra = pd.merge(compra,Precio_Producto, how="left", on=["IdProducto"])
    compra.sort_values("IdCompra")

    #Se reemplazan los 0 de precios2 por el valor de la media de los precios sin outliers
    compra.Precio2.fillna(CompraSinOut2.Precio.mean(),inplace = True)

    #compra.isnull().sum

    #Se reemplazan con 0 y 1 todos los valores de precios vacios y cantidades
    compra.Precio.fillna(0, inplace= True)
    compra.Cantidad.fillna(1, inplace= True)

    #se crea columna con unos
    compra['Outliers'] = np.ones(compra.shape[0])

    #Se marcan con 0 las filas que son outliers para reemplazarlas con precios 2 despues
    compra.loc[(compra['Precio']>=outliersSup),'Outliers'] = 0

    #Este for reemplaza los 0 y los outliers por el precio promedio sin outliers
    for i in range(0,compra.shape[0]):
        if compra.Precio[i] == 0 or compra.Precio[i]> outliersSup:
            compra.Precio[i] = compra.Precio2[i] #PRECIO2 ES EL PRECIO PROMEDIO SIN OUTLIERS


    compra.drop(columns=['Precio2'], inplace=True)
    #compra['Total venta'] = compra['Precio']*compra['Cantidad']

    #compra.isnull().sum()

    #tratamiento outliers cantidad
    #Se calcula el rango intercuartilico IQR.
    Q1 = compra["Cantidad"].quantile(0.25)
    Q3 = compra["Cantidad"].quantile(0.75)
    IQR = Q3 -Q1
    outliersSup = (Q3 + (1.5*IQR)) 

    # se crea un df sin outliers, se calcula la media de compras,y se imputan como valor en los casos que sean outliers.
    mask = compra["Cantidad"]<outliersSup
    compraCantSinOut = compra[mask]
    cantidadMedia = round(compraCantSinOut.Cantidad.mean())
    compra.loc[(compra['Cantidad'] >= outliersSup),"Cantidad"] = cantidadMedia #8
    compra["Compras_Totales"]= compra["Precio"]*compra["Cantidad"]
    compra.to_csv (r'C:/Users/notebook/OneDrive/Escritorio/Henry clase/DS-PI-ProyectoIndividual/Nuevosdataset/ComprasNor.csv', index = False, header=True)


In [12]:
CargaArchivoCompra()

Compra.csv


C:\Users\notebook\AppData\Local\Temp\ipykernel_6672\3997040176.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compra.Precio[i] = compra.Precio2[i] #PRECIO2 ES EL PRECIO PROMEDIO SIN OUTLIERS


### TABLA LOCALIDADES

In [13]:
def CargaArchivoLocalidades():
    #ds_path= pathlib.Path("Datasets")
    #print(ds_path)
    df_list=[]
    #archivo=glob.glob('.csv')
    #print("Hola")
    for file in glob.glob('Localida*.csv'):
        #print(file)
        df=pd.read_csv(file,delimiter = ',',encoding = "utf-8")
        df_list.append(df)
    global localidad
    localidad=pd.concat(df_list)
    #Tabla localidad
    #localidad.head()
    #localidad.isna().sum()

    #acomodo nombres
    localidad['nombre']=localidad.apply(lambda x: 'Ciudad de Buenos Aires' if x['municipio_nombre']=='Capital Federal' else x['nombre'], axis=1)

    #localidad.isna().sum()

    #corrijo nulos
    localidad["departamento_id"].fillna("Sin datos", inplace=True)
    localidad["departamento_nombre"].fillna("Sin datos", inplace=True)

    #Corrijo letras
    localidad["provincia_nombre"]=[x.replace('á',"a") for x in localidad["provincia_nombre"]]
    localidad["provincia_nombre"]=[x.replace('é',"e") for x in localidad["provincia_nombre"]]
    localidad["provincia_nombre"]=[x.replace('í',"i") for x in localidad["provincia_nombre"]]
    localidad["provincia_nombre"]=[x.replace('ó',"o") for x in localidad["provincia_nombre"]]
    localidad["provincia_nombre"]=[x.replace('ú',"u") for x in localidad["provincia_nombre"]]

    localidad["nombre"]=[x.replace('á',"a") for x in localidad["nombre"]]
    localidad["nombre"]=[x.replace('é',"e") for x in localidad["nombre"]]
    localidad["nombre"]=[x.replace('í',"i") for x in localidad["nombre"]]
    localidad["nombre"]=[x.replace('ó',"o") for x in localidad["nombre"]]
    localidad["nombre"]=[x.replace('ú',"u") for x in localidad["nombre"]]

    #Renombro columnas que tienen valores de Latitud y Longitud
    localidad= localidad.rename(columns= {"centroide_lat":"Latitud"})
    localidad= localidad.rename(columns= {"centroide_lon":"Longitud"})

    #Creo dataframe de provincia
    provincias['Provincia'] = localidad.provincia_nombre
    provincias['IdProvincia'] = localidad.provincia_id

    #Borro duplicados
    provincias.drop_duplicates(inplace=True)

    #Renombro columnas
    localidad= localidad.rename(columns= {"provincia_nombre":"Provincia"})

    #Borro columnas
    localidad.drop(columns='departamento_nombre',inplace=True)

    #Capitalizo columna nombre
    localidad["nombre"]= localidad["nombre"].str.title()

    #Cambio nombre de las columnas
    localidad= localidad.rename(columns= {"id":"IdLocalidad", "nombre":"Localidad","provincia_id":"IdProvincia", "departamento_id":"IdDepartamento"})

    #Borroduplicados
    localidad.drop_duplicates(subset=["Localidad","IdProvincia"], inplace=True)
    localidad.reset_index(inplace=True)

    #Borro columnas
    localidad.drop(['categoria','fuente', 'localidad_censal_id', 'localidad_censal_nombre', 'municipio_id', 'municipio_nombre', 'IdDepartamento'], axis = 'columns', inplace=True)

    #Creo nueva fila nula
    localidad=localidad.append({'Latitud' : 0 , 'Longitud' : 0, 'IdLocalidad' : 0, 'Localidad':"Sin Datos","Provincia":"Sin Datos"} , ignore_index=True)
    
    localidad.to_csv (r'C:/Users/notebook/OneDrive/Escritorio/Henry clase/DS-PI-ProyectoIndividual/Nuevosdataset/LocalidadesNor.csv', index = False, header=True)
    provincias.to_csv (r'C:/Users/notebook/OneDrive/Escritorio/Henry clase/DS-PI-ProyectoIndividual/Nuevosdataset/ProvinciasNor.csv', index = False, header=True)
    

In [14]:
CargaArchivoLocalidades()

C:\Users\notebook\AppData\Local\Temp\ipykernel_6672\4288571138.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  localidad=localidad.append({'Latitud' : 0 , 'Longitud' : 0, 'IdLocalidad' : 0, 'Localidad':"Sin Datos","Provincia":"Sin Datos"} , ignore_index=True)


### TABLA CLIENTES

In [15]:
def CargaArchivoClientes():
    #ds_path= pathlib.Path("Datasets")
    #print(ds_path)
    df_list=[]
    #archivo=glob.glob('.csv')
    #print("Hola")
    for file in glob.glob('Client*.csv'):
        #print(file)
        df=pd.read_csv(file,delimiter = ';',encoding = "utf-8", usecols=["ID","Provincia","Nombre_y_Apellido","Domicilio","Telefono","Edad","Localidad","X","Y","col10"])
        df_list.append(df)
    cliente=pd.concat(df_list)
    #cliente.head()
    #cliente.info()
    #cliente.shape
    #cliente.isnull().sum()

    #Drop col10 que tiene todos los valores vacios
    cliente.drop(columns= 'col10', inplace = True)

    #corrijo nombres
    cliente= cliente.rename(columns= {"ID":"IdCliente"})
    cliente= cliente.rename(columns= {"X":"Longitud"})
    cliente= cliente.rename(columns= {"Y":"Latitud"})

    #corrijo mayuscula
    cliente["Nombre_y_Apellido"]= cliente["Nombre_y_Apellido"].str.title()
    cliente["Domicilio"]= cliente["Domicilio"].str.title()
    cliente["Localidad"]= cliente["Localidad"].str.title()

    #reemplazo faltantes
    cliente.Provincia.fillna("Sin Datos", inplace=True)
    cliente.Nombre_y_Apellido.fillna("Sin Datos", inplace=True)
    cliente.Localidad.fillna("Sin Datos", inplace=True)
    cliente.Telefono.fillna("Sin Datos", inplace=True)
    cliente.Domicilio.fillna("Sin Datos", inplace=True)

    #cliente.isnull().sum()

    #reemplazo coma por punto
    cliente["Longitud"] = cliente["Longitud"].str.replace(",", ".")
    cliente["Latitud"] = cliente["Latitud"].str.replace(",", ".")

    #reemplazo Latitud y Longitud que son nulos con ceros
    cliente.Latitud.fillna(0.0, inplace=True)
    cliente.Longitud.fillna(0.0, inplace=True)

    #normalizo provincia de Buenos Aires
    cliente["Provincia"]=cliente.apply(lambda x: "Buenos Aires" if x["Provincia"]=="Ciudad de Buenos Aires"  else x["Provincia"], axis=1)

    #acomodo acentos
    cliente["Provincia"]=[x.replace('á',"a") for x in cliente["Provincia"]]
    cliente["Provincia"]=[x.replace('é',"e") for x in cliente["Provincia"]]
    cliente["Provincia"]=[x.replace('í',"i") for x in cliente["Provincia"]]
    cliente["Provincia"]=[x.replace('ó',"o") for x in cliente["Provincia"]]
    cliente["Provincia"]=[x.replace('ú',"u") for x in cliente["Provincia"]]
    cliente["Localidad"]=[x.replace('á',"a") for x in cliente["Localidad"]]
    cliente["Localidad"]=[x.replace('é',"e") for x in cliente["Localidad"]]
    cliente["Localidad"]=[x.replace('í',"i") for x in cliente["Localidad"]]
    cliente["Localidad"]=[x.replace('ó',"o") for x in cliente["Localidad"]]
    cliente["Localidad"]=[x.replace('ú',"u") for x in cliente["Localidad"]]

    #cliente.isnull().sum()

    #merge entre df Localidades y Clientes para traer dato IdLocalidad a Clientes 
    cliente = pd.merge(cliente,localidad, how="left", on=["Provincia","Localidad"])


    #reemplazo Latitud y Longitud sin datos con Latitud y Longitud de tabla Localidades 
    cliente["Longitud_x"]=cliente.apply(lambda x: x["Longitud_y"] if x["Longitud_x"]==0  else x["Longitud_x"], axis=1)
    cliente["Latitud_x"]=cliente.apply(lambda x: x["Latitud_y"] if x["Latitud_x"]==0  else x["Latitud_x"], axis=1)

    #cliente.isnull().sum()

    #renombro columnas
    cliente= cliente.rename(columns= {"Longitud_x":"Longitud"})
    cliente= cliente.rename(columns= {"Latitud_x":"Latitud"})

    #cambio tipo dato Latitud y Longitud
    cliente["Latitud"]= cliente.Latitud.astype(float)
    cliente["Longitud"]= cliente.Longitud.astype(float)

    #elimino columnas que se agregaron en el merge
    cliente.drop(['index'], axis = 'columns', inplace=True)


    #funcion para encontrar Localidades por similitud - Se recupera IdLocalidad y IdProvincia
    def verificarLocalidadClientes(row):
        ratiomayor=0
        for i in range(0,localidad.shape[0]): 

                Ratio= fuzz.token_sort_ratio(row["Localidad"].lower(), localidad.Localidad[i].lower())
                if Ratio>ratiomayor:
                        ratiomayor=Ratio
                        miloc=localidad.Localidad[i]
                        id=localidad.IdLocalidad[i]
                        idprov=localidad.IdProvincia[i]

        row["Localidad"]=miloc
        row["IdLocalidad"]=id
        row["IdProvincia"]=idprov


        return row

    #cliente.isnull().sum()

    #creo DF de cliente con IdLocalidad faltante 
    clientes_nulos= cliente.loc[cliente['IdLocalidad'].isnull() == True].copy()

    #Aplico funcion
    clientes_nulos= clientes_nulos.apply(verificarLocalidadClientes,axis=1)

    #Borro columnas innecesarias para el merge 
    clientes_nulos.drop(columns=["Latitud_y","Longitud_y","Domicilio","Telefono","Edad"],inplace=True)

    #cliente.isnull().sum()

    #merge de Clientes con DF Clientes con IDlocalidad Nulos corregido
    cliente = pd.merge(cliente,clientes_nulos, how="left", on=["IdCliente"])

    #completo IdLocalidad en tabla cliente original con 0 para prepararlos para el reemplazo
    cliente["IdLocalidad_x"]= cliente["IdLocalidad_x"].fillna(0)
    cliente["IdProvincia_x"]= cliente["IdProvincia_x"].fillna(0)

    #realizo reemplazo de IdLocalidad y IdProvincia traidos con el merge
    cliente["IdLocalidad_x"]=cliente.apply(lambda x: x["IdLocalidad_y"] if x["IdLocalidad_x"]==0  else x["IdLocalidad_x"], axis=1)
    cliente["IdProvincia_x"]=cliente.apply(lambda x: x["IdProvincia_y"] if x["IdProvincia_x"]==0  else x["IdProvincia_x"], axis=1)

    #25 Localidades sin IdLocalidad - se lo completa como sin dato
    cliente["IdLocalidad_x"]=cliente["IdLocalidad_x"].fillna(0)

    #Elimino columnas innecesarias
    cliente.drop(['Latitud_y','Longitud_y','Provincia_y','Nombre_y_Apellido_y','Localidad_y', 'Longitud_y', 'IdLocalidad_y','IdProvincia_y'], axis = 'columns', inplace=True)

    #Renombro columnas
    cliente= cliente.rename(columns= {"Provincia_x":"Provincia"})
    cliente= cliente.rename(columns= {"Nombre_y_Apellido_x":"Nombre_y_Apellido"})
    cliente= cliente.rename(columns= {"Localidad_x":"Localidad"})
    cliente= cliente.rename(columns= {"Longitud_x":"Longitud"})
    cliente= cliente.rename(columns= {"Latitud_x":"Latitud"})
    cliente= cliente.rename(columns= {"IdLocalidad_x":"IdLocalidad"})
    cliente= cliente.rename(columns= {"IdProvincia_x":"IdProvincia"})

    #cliente.isnull().sum()

    #Borro nulos restantes de Latitud
    cliente.Latitud.fillna("0.0", inplace=True)
    
    cliente.to_csv (r'C:/Users/notebook/OneDrive/Escritorio/Henry clase/DS-PI-ProyectoIndividual/Nuevosdataset/ClientesNor.csv', index = False, header=True)

In [16]:
CargaArchivoClientes()

### TABLA PROVEEDORES

In [17]:
def CargaArchivoProveedores():
    #ds_path= pathlib.Path("Datasets")
    #print(ds_path)
    df_list=[]
    #archivo=glob.glob('.csv')
    #print("Hola")
    for file in glob.glob('Prov*.csv'):
        #print(file)
        df=pd.read_csv(file,delimiter = ',',encoding = "Ansi")
        df_list.append(df)
    proveedor=pd.concat(df_list)
    #proveedor.head()

    #Renombro columnas
    proveedor = proveedor.rename(columns= {'IDProveedor':'IdProveedor', 'Nombre':'Proveedor','Address':'Direccion', 'City':'Localidad', 'State':'Provincia', 'Country':'Pais', 'departamen':'Departamento'})

    #Corrijo letras
    proveedor["Provincia"]=[x.replace('á',"a") for x in proveedor["Provincia"]]
    proveedor["Provincia"]=[x.replace('é',"e") for x in proveedor["Provincia"]]
    proveedor["Provincia"]=[x.replace('í',"i") for x in proveedor["Provincia"]]
    proveedor["Provincia"]=[x.replace('ó',"o") for x in proveedor["Provincia"]]
    proveedor["Provincia"]=[x.replace('ú',"u") for x in proveedor["Provincia"]]

    proveedor["Localidad"]=[x.replace('á',"a") for x in proveedor["Localidad"]]
    proveedor["Localidad"]=[x.replace('é',"e") for x in proveedor["Localidad"]]
    proveedor["Localidad"]=[x.replace('í',"i") for x in proveedor["Localidad"]]
    proveedor["Localidad"]=[x.replace('ó',"o") for x in proveedor["Localidad"]]
    proveedor["Localidad"]=[x.replace('ú',"u") for x in proveedor["Localidad"]]

    #cliente.isna().sum()
    #Llena datos vacios
    proveedor.Proveedor.fillna('Sin Datos', inplace= True)

    #Capitalizo columnas
    proveedor['Direccion'] = proveedor['Direccion'].str.title()
    proveedor['Localidad'] = proveedor['Localidad'].str.title()
    proveedor['Provincia'] = proveedor['Provincia'].str.title()
    proveedor['Pais'] = proveedor['Pais'].str.title()
    proveedor['Departamento'] = proveedor['Departamento'].str.title()

    provincias.reset_index(inplace=True)

    #normalizo provincia 
    lista_provincia=[]
    for i in range(provincias.shape[0]):
        lista_provincia.append(provincias.loc[i,"Provincia"])

    #Funcion para encontrar parecidos
    def verificarProvinciaProveedores(row):
        if(row["Provincia"] not in lista_provincia):
                ratiomayor=0
                for i in range(0,provincias.shape[0]): 
                    Ratio= fuzz.token_sort_ratio(row["Provincia"].lower(), provincias.Provincia[i].lower())
                    if Ratio>ratiomayor:
                            ratiomayor=Ratio
                            miloc=provincias.Provincia[i]
                row["Provincia"]=miloc
        return row

    #Corrijo CABA
    proveedor['Provincia']=proveedor.apply(lambda x: 'Buenos Aires' if x['Provincia']=='CABA' else x['Provincia'],axis=1)

    #Aplico funcion
    proveedor=proveedor.apply(verificarProvinciaProveedores, axis=1)

    #normalizo ciudad de buenos aires asi porque con la funcion no me la normaliza
    ver_loc = list(proveedor['Localidad'].unique())
    #for n in ver_loc:
        #print(n)

    #creamos una lista con las denominaciones más comunes que puede recibir la ciudad de buenos aires
    list_BSAS = ["Ciudad de Buenos Aires", "Capital Federal", "Cap Fed", "CABA"]
    nom_normalizado = [ ]
    loc1 = " "
    loc2 = " "
    for m in list_BSAS:
        loc1 = m
        for n in ver_loc:
            loc2 = n
            Ratio = fuzz.ratio(loc1.lower(),loc2.lower())
            Partial_Ratio = fuzz.partial_ratio(loc1.lower(),loc2.lower())
            Token_Sort_Ratio = fuzz.token_sort_ratio(loc1, loc2)
            Token_Set_Ratio = fuzz.token_set_ratio(loc1, loc2)
            if ((Partial_Ratio > 70) and (Token_Set_Ratio > 70)):
                nom_normalizado.append(loc2)
    print(nom_normalizado)

    for n in range (len(nom_normalizado)):
        proveedor['Localidad'].loc[(proveedor['Localidad']==str(nom_normalizado[n]))]='Ciudad de Buenos Aires'

    #normalizacion de localidad
    lista_localidades=[]
    for i in range(localidad.shape[0]):
        lista_localidades.append(localidad.loc[i,"Localidad"])

    def verificarLocalidadProveedor(row):
        if(row["Localidad"] not in lista_localidades):
                ratiomayor=0
                for i in range(0,localidad.shape[0]): 
                    Ratio= fuzz.token_sort_ratio(row["Localidad"].lower(), localidad.Localidad[i].lower())
                    if Ratio>ratiomayor:
                            ratiomayor=Ratio
                            miloc=localidad.Localidad[i]
                row["Localidad"]=miloc
        return row

    proveedor=proveedor.apply(verificarLocalidadProveedor, axis=1)

    #traigo idlocalidad con el merge
    proveedor = pd.merge(proveedor,localidad, how="left", on=["Localidad","Provincia"])
    proveedor.drop(['Pais','Departamento','index','Latitud','Longitud','Localidad','Provincia'], axis = 'columns', inplace=True)

    #proveedor.isnull().sum()

    proveedor.to_csv (r'C:/Users/notebook/OneDrive/Escritorio/Henry clase/DS-PI-ProyectoIndividual/Nuevosdataset/PreveedoresNor.csv', index = False, header=True)

In [18]:
CargaArchivoProveedores()

[]


### TABLA SUCURSAL

In [19]:
def CargaArchivoSucursal():
    #ds_path= pathlib.Path("Datasets")
    #print(ds_path)
    df_list=[]
    #archivo=glob.glob('.csv')
    #print("Hola")
    for file in glob.glob('Sucursa*.csv'):
        print(file)
        df=pd.read_csv(file,delimiter = ';',encoding = "utf-8")
        df_list.append(df)
    sucursal=pd.concat(df_list)
    #sucursal.head()

    #Renombro columnas
    sucursal = sucursal.rename(columns= {'ID':'IdSucursal'})

    #Corrijo letras
    sucursal["Provincia"]=[x.replace('á',"a") for x in sucursal["Provincia"]]
    sucursal["Provincia"]=[x.replace('é',"e") for x in sucursal["Provincia"]]
    sucursal["Provincia"]=[x.replace('í',"i") for x in sucursal["Provincia"]]
    sucursal["Provincia"]=[x.replace('ó',"o") for x in sucursal["Provincia"]]
    sucursal["Provincia"]=[x.replace('ú',"u") for x in sucursal["Provincia"]]

    sucursal["Localidad"]=[x.replace('á',"a") for x in sucursal["Localidad"]]
    sucursal["Localidad"]=[x.replace('é',"e") for x in sucursal["Localidad"]]
    sucursal["Localidad"]=[x.replace('í',"i") for x in sucursal["Localidad"]]
    sucursal["Localidad"]=[x.replace('ó',"o") for x in sucursal["Localidad"]]
    sucursal["Localidad"]=[x.replace('ú',"u") for x in sucursal["Localidad"]]

    #modificamos el tipo de dato de la columna longitud
    sucursal["Longitud"] = sucursal["Longitud"].str.replace(",", ".").astype(float)
    sucursal["Latitud"] = sucursal["Latitud"].str.replace(",", ".").astype(float)

    #sucursal.isnull().sum()

    #normalizacion de provincia
    lista_provincia=[]
    for i in range(provincias.shape[0]):
        lista_provincia.append(provincias.loc[i,"Provincia"])

    #Funcion para encontrar parecios
    def verificarProvincia(row):
        if(row["Provincia"] not in lista_provincia):
                ratiomayor=0
                for i in range(0,provincias.shape[0]): 
                    Ratio= fuzz.token_sort_ratio(row["Provincia"].lower(), provincias.Provincia[i].lower())
                    if Ratio>ratiomayor:
                            ratiomayor=Ratio
                            miprov=provincias.Provincia[i]
                row["Provincia"]=miprov
        return row

    #Corrijo CABA 
    sucursal['Provincia']=sucursal.apply(lambda x: 'Buenos Aires' if x['Provincia']=='CABA' else x['Provincia'],axis=1)

    #Aplico funcion
    sucursal=sucursal.apply(verificarProvincia, axis=1)

    #normalizo ciudad de buenos aires asi porque con la funcion no me la normaliza
    ver_loc = list(sucursal['Localidad'].unique())
    #for n in ver_loc:
        #print(n)

    # Creamos una lista con las denominaciones más comunes que puede recibir la ciudad de buenos aires
    list_BSAS = ["Ciudad de Buenos Aires", "Capital Federal", "Cap Fed", "CABA"]
    nom_normalizado = [ ]
    loc1 = " "
    loc2 = " "
    for m in list_BSAS:
        loc1 = m
        for n in ver_loc:
            loc2 = n
            Ratio = fuzz.ratio(loc1.lower(),loc2.lower())
            Partial_Ratio = fuzz.partial_ratio(loc1.lower(),loc2.lower())
            Token_Sort_Ratio = fuzz.token_sort_ratio(loc1, loc2)
            Token_Set_Ratio = fuzz.token_set_ratio(loc1, loc2)
            #print(f"* {loc1} vs {loc2}")
            #print(Ratio)
            #print(Partial_Ratio)
            #print(Token_Sort_Ratio)
            #print(Token_Set_Ratio, "\n")
            if ((Partial_Ratio > 70) and (Token_Set_Ratio > 70)):
                nom_normalizado.append(loc2)
    print(nom_normalizado)

    for n in range (len(nom_normalizado)):
        sucursal['Localidad'].loc[(sucursal['Localidad']==str(nom_normalizado[n]))]='Ciudad de Buenos Aires'

    #normalizacion de localidad
    lista_localidades=[]
    for i in range(localidad.shape[0]):
        lista_localidades.append(localidad.loc[i,"Localidad"])

    def verificarLocalidad(row):
        if(row["Localidad"] not in lista_localidades):
                ratiomayor=0
                for i in range(0,localidad.shape[0]): 
                    Ratio= fuzz.token_sort_ratio(row["Localidad"].lower(), localidad.Localidad[i].lower())
                    if Ratio>ratiomayor:
                            ratiomayor=Ratio
                            miloc=localidad.Localidad[i]
                row["Localidad"]=miloc
        return row

    sucursal=sucursal.apply(verificarLocalidad, axis=1)

    #traigo idlocalidad con el merge
    sucursal = pd.merge(sucursal,localidad, how="left", on=["Localidad","Provincia"])

    sucursal.drop(['Latitud_y','Longitud_y', 'Localidad', 'Provincia', 'index', 'Latitud_x', 'Longitud_x',], axis = 'columns', inplace=True)

    #localidad.isnull().sum()
    sucursal.to_csv (r'C:/Users/notebook/OneDrive/Escritorio/Henry clase/DS-PI-ProyectoIndividual/Nuevosdataset/SucursalesNor.csv', index = False, header=True)

In [20]:
CargaArchivoSucursal()

Sucursales.csv
['Ciudad de Buenos Aires', 'Cdad de Buenos Aires', 'Capital', 'Capital Federal', 'Cap.   Federal', 'CapFed', 'Cap. Fed.', 'CABA']


C:\Users\notebook\AppData\Local\Temp\ipykernel_6672\37095291.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sucursal['Localidad'].loc[(sucursal['Localidad']==str(nom_normalizado[n]))]='Ciudad de Buenos Aires'
